# In the main branch, this file should be no executed and with no parameters set.
# In the development branch, it must be fully executed and must include tests.

GOAL: calculate the physical quantities (magnetization, susceptibility, binder cumulant, and possible errors) from the raw results stored at an SQLite DB.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

## Parameters

In [ ]:
# Set the filename of the raw results database.
# By raw I mean coming directly from simulations.
#
# Example: 'bubble_filtering__8_neighbors_squared_network.db'

db_filename = 

<br>
Hereafter, the code should not be modified at the standard usage.

## Functions

In [ ]:
def calc_averages(columnLabel, results_df):
    '''
    Calculate the means for grouped values of columnLabel.
    
    Parameters:
    columnLabel should be in ['mT', 'm2T', 'm4T'].
    results_df contains the temporal averages.
    
    Returns a pandas series of replications and networks averages.
    '''
    
    # Do the mean over the replications
    x = pd.DataFrame(results_df.groupby(['v','N','q','net'])[columnLabel].mean())
    x.reset_index(inplace=True)
    
    # Do the mean over the network configuration
    x = x.groupby(['v','N','q'])[columnLabel].mean()
    
    
    return x

## Main

In [ ]:
# Read the database with pandas

conn = sqlite3.connect('../results_databases/' + db_filename)

df = pd.read_sql('SELECT * FROM temporal_mean_results', conn,
                 index_col = 'index'
                )
df.sort_values(['v','N','q','net'], inplace=True)

conn.close()

In [ ]:
df.head(20)

In [ ]:
# Calculate the physical interesting quantities

phys_quant = pd.DataFrame(calc_averages('mT', df))
phys_quant.reset_index(inplace=True)
phys_quant.rename({'mT': 'M'},axis=1,inplace=True)

m2df = calc_averages('m2T', df)
m4df = calc_averages('m4T', df)

phys_quant['M_err'] = np.array(m2df) - phys_quant['M']**2
phys_quant['M_noMod'] = np.array(calc_averages('mT_noMod', df))
phys_quant['M_noMod_err'] = np.array(m2df) - phys_quant['M_noMod']**2

phys_quant['X'] = phys_quant['N']*phys_quant['M_err']
phys_quant['X_noMod'] = phys_quant['N']*phys_quant['M_noMod_err']

phys_quant['U'] = 1 - np.array(m4df)/(3*np.array(m2df)**2)

phys_quant['u'] = -np.log(1-3*phys_quant['U']/2)

In [ ]:
phys_quant.head(20)

In [ ]:
# Persist the data frame

# Comment the exception raising to persist the data frame
raise Exception('Do not execute this cell accidentally.')

phys_quant.to_csv(
    '../results_databases/' + db_filename[:-3] + '__MXU.csv')